In [ ]:
!pip install kaggle
!pip install fastai
!pip install sklearn
!pip install transformers
!pip install datasets
!pip install sentencepiece

In [ ]:
import pandas as pd

df = pd.read_csv('train.csv')
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor
df

In [38]:
from datasets import Dataset, DatasetDict

# Transformers uses a Dataset object for storing a dataset
ds = Dataset.from_pandas(df)
ds

model_nm = 'microsoft/deberta-v3-small'


In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer

tokz = AutoTokenizer.from_pretrained(model_nm)
tokz.tokenize("G'day folks, I'm Jeremy from fast.ai!")


In [ ]:
def tok_func(x):
  return tokz(x["input"])

# to run it in parallel and fast
tok_ds = ds.map(tok_func, batched=True)

row = tok_ds[0]
row['input'], row['input_ids']

# to prepare the labels for the transformers, we rename the column as labels
tok_ds = tok_ds.rename_columns({'score':'labels'})



In [ ]:
eval_df = pd.read_csv('test.csv')
eval_df.describe()


In [ ]:
from sklearn.model.selection import train_test_split
#validation set
dds = tok_ds.train_test_split(0.25, seed=42)
print(dds)

#test set
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)



In [ ]:
def corr_d(eval_pred):
  return {'pearson': corr(*eval_pred)}


In [ ]:
# trainnig the model
def corr_d(eval_pred):
  return {'pearson': corr(*eval_pred)}

bs = 128
epochs = 4
lr = 8e-5

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)


trainer.train();


In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
print(preds)

preds = np.clip(preds, 0, 1)
print(preds)


In [ ]:
#visulizing the predictions

from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

# Scatter plot
plt.scatter(data[:, 0], data[:, 1], c=data[:, 2], cmap='hot')

# Display the plot
plt.show()
